# Deep Recommender: Behavior Sequence Transformer (BST)

This notebook demonstrates how recommendations can be made based on interaction sequences using transformer-based model.

### Use Case
We assume hybrid data that can include user features like age, item features like category, and interaction event sequences for certain user-item pairs. Each interaction event can be associated with additional features such as rating and timestamp. Out goal is to build a model that, for each user, predicts the item this user is likely ot interact with.

### Prototype: Approach and Data
The prototype is based on Behavior Sequence Transformer (BST) design [1] and Keras implementation created by Khalid Salama's [2]. We use `MovieLens 1M` dataset. See `datasets.md` for details.

### Usage and Productization
This prototype can be used to evaluate the feasibility of the transformer based approach for a specific dataset. Its performance can also be compared with other prototypes and optimal approach can be selected.

### References:
1. Chen Q., Zhao H., Li W., Huang P., and Ou W. -- Behavior Sequence Transformer for E-commerce Recommendation in Alibaba, 2019
2. Salama K. -- A Transformer-based recommendation system, 2020

In [8]:
#
# Imports and settings
#
import math
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import StringLookup

from sklearn.model_selection import train_test_split
from tabulate import tabulate

import matplotlib.pyplot as plt
plt.rcParams.update({'pdf.fonttype': 'truetype'})
warnings.simplefilter("ignore")

print("Keras version " + tf.keras.__version__)
print("Tensorflow version " + tf.__version__)

Keras version 2.6.0
Tensorflow version 2.6.0


In [2]:
#
# Helper functions
#
def print_df(df, col_width = 10, rows = 10, max_cols = 10):
    def short_srt(x):
        return x if len(x) < col_width else x[:col_width-3] + "..."
    df_short = df.head(rows).applymap(lambda x: short_srt(str(x)))
    
    if len(df_short.columns) > max_cols:
        df_short = df_short.iloc[:, 0:max_cols-1]
        df_short['...'] = '...'
    
    print(tabulate(df_short, headers='keys', tablefmt='psql'))
    print(f'{len(df)} rows x {len(df.columns)} columns')

# Data Loading

In [3]:
def load_moviewlens_1m():
    base_data_path = '~/ALGO/tensor-house-data-large-unpacked/movielens-1m/'

    users = pd.read_csv(
        base_data_path + "users.dat",
        sep="::",
        names=["user_id", "sex", "age_group", "occupation", "zip_code"],
        encoding='ISO-8859-1'
    )

    ratings = pd.read_csv(
        base_data_path + "ratings.dat",
        sep="::",
        names=["user_id", "movie_id", "rating", "unix_timestamp"],
        encoding='ISO-8859-1'
    )

    movies = pd.read_csv(
        base_data_path + "movies.dat", 
        sep="::", 
        names=["movie_id", "title", "genres"],
        encoding='ISO-8859-1'
    )
    
    return users, ratings, movies

# Baseline: Neural Collaborating Filtering

We start with implementing a baseline model that uses only the rating data. We use a basic factorization network that maps user and item IDs to embeddings and estimates the rating as a dot product.

In [ ]:
#
# Load data
#
users, ratings, movies = load_moviewlens_1m()

#
# Train/test split
#
n = ratings.shape[0]
train_idx, test_idx, _, _ = train_test_split(np.arange(n), np.arange(n), test_size=0.20, random_state=42)
x_train, x_test = ratings[['user_id', 'movie_id']].values[train_idx], ratings[['user_id', 'movie_id']].values[test_idx]
y_train, y_test = ratings['rating'].values[train_idx], ratings['rating'].values[test_idx]

max_user_id, max_item_id = max(ratings['user_id']), max(ratings['movie_id'])

#
# Model specification
#
embedding_dim = 8

input_user = layers.Input(shape=(1,))
input_item = layers.Input(shape=(1,))

embedding_user = layers.Embedding(input_dim=max_user_id + 1, output_dim=embedding_dim)(input_user)
embedding_item = layers.Embedding(input_dim=max_item_id + 1, output_dim=embedding_dim)(input_item)

combined = layers.Dot(axes=2)([embedding_user, embedding_item])

score = layers.Dense(1, activation='linear')(combined)

model_sgd = keras.models.Model(inputs=[input_user, input_item], outputs=score)

#
# Model training
#
opt = keras.optimizers.Adam(learning_rate=0.01)
model_sgd.compile(optimizer=opt, loss='mse', metrics=[keras.metrics.MeanAbsoluteError()]) 
history_sgd = model_sgd.fit([x_train[:, 0], x_train[:, 1]], y_train, 
                    batch_size=256, epochs=8, verbose=1, 
                    validation_data=([x_test[:, 0], x_test[:, 1]], y_test)) 

# BST: Data Preparation

We do the initial data preparation, split the dataset into train/test sets, and save them into separate files.

In [4]:
#
# Load data
#
users, ratings, movies = load_moviewlens_1m()

users["user_id"] = users["user_id"].apply(lambda x: f"user_{x}")
users["age_group"] = users["age_group"].apply(lambda x: f"group_{x}")
users["occupation"] = users["occupation"].apply(lambda x: f"occupation_{x}")

movies["movie_id"] = movies["movie_id"].apply(lambda x: f"movie_{x}")

ratings["movie_id"] = ratings["movie_id"].apply(lambda x: f"movie_{x}")
ratings["user_id"] = ratings["user_id"].apply(lambda x: f"user_{x}")
ratings["rating"] = ratings["rating"].apply(lambda x: float(x))

genres = [
    "Action",
    "Adventure",
    "Animation",
    "Children's",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    "Film-Noir",
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western",
]

for genre in genres:
    movies[genre] = movies["genres"].apply(
        lambda values: int(genre in values.split("|"))
    )
    
#
# Transform the movie ratings data into sequences
#
ratings_group = ratings.sort_values(by=["unix_timestamp"]).groupby("user_id")

ratings_data = pd.DataFrame(
    data={
        "user_id": list(ratings_group.groups.keys()),
        "movie_ids": list(ratings_group.movie_id.apply(list)),
        "ratings": list(ratings_group.rating.apply(list)),
        "timestamps": list(ratings_group.unix_timestamp.apply(list)),
    }
)

sequence_length = 8
step_size = 1

def create_sequences(values, window_size, step_size):
    sequences = []
    start_index = 0
    while True:
        end_index = start_index + window_size
        seq = values[start_index:end_index]
        if len(seq) < window_size:
            seq = values[-window_size:]
            if len(seq) == window_size:
                sequences.append(seq)
            break
        sequences.append(seq)
        start_index += step_size
    return sequences


ratings_data.movie_ids = ratings_data.movie_ids.apply(
    lambda ids: create_sequences(ids, sequence_length, step_size)
)

ratings_data.ratings = ratings_data.ratings.apply(
    lambda ids: create_sequences(ids, sequence_length, step_size)
)

del ratings_data["timestamps"]

ratings_data_movies = ratings_data[["user_id", "movie_ids"]].explode(
    "movie_ids", ignore_index=True
)
ratings_data_rating = ratings_data[["ratings"]].explode("ratings", ignore_index=True)
ratings_data_transformed = pd.concat([ratings_data_movies, ratings_data_rating], axis=1)
ratings_data_transformed = ratings_data_transformed.join(
    users.set_index("user_id"), on="user_id"
)
ratings_data_transformed.movie_ids = ratings_data_transformed.movie_ids.apply(
    lambda x: ",".join(x)
)
ratings_data_transformed.ratings = ratings_data_transformed.ratings.apply(
    lambda x: ",".join([str(v) for v in x])
)

del ratings_data_transformed["zip_code"]

ratings_data_transformed.rename(
    columns={"movie_ids": "sequence_movie_ids", "ratings": "sequence_ratings"},
    inplace=True,
)

In [14]:
print_df(ratings_data_transformed, col_width=35)

+----+-----------+-------------------------------------+---------------------------------+-------+-------------+---------------+
|    | user_id   | sequence_movie_ids                  | sequence_ratings                | sex   | age_group   | occupation    |
|----+-----------+-------------------------------------+---------------------------------+-------+-------------+---------------|
|  0 | user_1    | movie_3186,movie_1721,movie_1270... | 4.0,4.0,5.0,5.0,3.0,5.0,4.0,4.0 | F     | group_1     | occupation_10 |
|  1 | user_1    | movie_1721,movie_1270,movie_1022... | 4.0,5.0,5.0,3.0,5.0,4.0,4.0,5.0 | F     | group_1     | occupation_10 |
|  2 | user_1    | movie_1270,movie_1022,movie_2340... | 5.0,5.0,3.0,5.0,4.0,4.0,5.0,4.0 | F     | group_1     | occupation_10 |
|  3 | user_1    | movie_1022,movie_2340,movie_1836... | 5.0,3.0,5.0,4.0,4.0,5.0,4.0,3.0 | F     | group_1     | occupation_10 |
|  4 | user_1    | movie_2340,movie_1836,movie_3408... | 3.0,5.0,4.0,4.0,5.0,4.0,3.0,5.0 | F     

In [7]:
#
# Save train/test datasets to files
#
random_selection = np.random.rand(len(ratings_data_transformed.index)) <= 0.80
train_data = ratings_data_transformed[random_selection]
test_data = ratings_data_transformed[~random_selection]

train_data.to_csv("train_data.csv", index=False, sep="|", header=False)
test_data.to_csv("test_data.csv", index=False, sep="|", header=False)

# BST: Feature Encoding 

Next, we map movie IDs and associated genres using embedding layers to prepare the input for the transformer. 

User features are mapped separately and will further be concatenated with the output of the transformer. 

In [8]:
CSV_HEADER = list(ratings_data_transformed.columns)
USER_FEATURES = ["sex", "age_group", "occupation"]
MOVIE_FEATURES = ["genres"]
CATEGORICAL_FEATURES_WITH_VOCABULARY = {
    "user_id": list(users.user_id.unique()),
    "movie_id": list(movies.movie_id.unique()),
    "sex": list(users.sex.unique()),
    "age_group": list(users.age_group.unique()),
    "occupation": list(users.occupation.unique()),
}


def get_dataset_from_csv(csv_file_path, shuffle=False, batch_size=128):
    def process(features):
        movie_ids_string = features["sequence_movie_ids"]
        sequence_movie_ids = tf.strings.split(movie_ids_string, ",").to_tensor()

        # The last movie id in the sequence is the target movie
        features["target_movie_id"] = sequence_movie_ids[:, -1]
        features["sequence_movie_ids"] = sequence_movie_ids[:, :-1]

        ratings_string = features["sequence_ratings"]
        sequence_ratings = tf.strings.to_number(
            tf.strings.split(ratings_string, ","), tf.dtypes.float32
        ).to_tensor()

        # The last rating in the sequence is the target for the model to predict
        target = sequence_ratings[:, -1]
        features["sequence_ratings"] = sequence_ratings[:, :-1]

        return features, target

    dataset = tf.data.experimental.make_csv_dataset(
        csv_file_path,
        batch_size=batch_size,
        column_names=CSV_HEADER,
        num_epochs=1,
        header=False,
        field_delim="|",
        shuffle=shuffle,
    ).map(process)

    return dataset


def create_model_inputs():
    return {
        "user_id": layers.Input(name="user_id", shape=(1,), dtype=tf.string),
        "sequence_movie_ids": layers.Input(
            name="sequence_movie_ids", shape=(sequence_length - 1,), dtype=tf.string
        ),
        "target_movie_id": layers.Input(
            name="target_movie_id", shape=(1,), dtype=tf.string
        ),
        "sequence_ratings": layers.Input(
            name="sequence_ratings", shape=(sequence_length - 1,), dtype=tf.float32
        ),
        "sex": layers.Input(name="sex", shape=(1,), dtype=tf.string),
        "age_group": layers.Input(name="age_group", shape=(1,), dtype=tf.string),
        "occupation": layers.Input(name="occupation", shape=(1,), dtype=tf.string),
    }


def encode_input_features(
    inputs,
    include_user_id=True,
    include_user_features=True,
    include_movie_features=True,
):

    encoded_transformer_features = []
    encoded_other_features = []

    other_feature_names = []
    if include_user_id:
        other_feature_names.append("user_id")
    if include_user_features:
        other_feature_names.extend(USER_FEATURES)

    ## Encode user features
    for feature_name in other_feature_names:
        # Convert the string input values into integer indices
        vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]
        idx = StringLookup(vocabulary=vocabulary, mask_token=None, num_oov_indices=0)(
            inputs[feature_name]
        )
        # Compute embedding dimensions
        embedding_dims = int(math.sqrt(len(vocabulary)))
        # Create an embedding layer with the specified dimensions
        embedding_encoder = layers.Embedding(
            input_dim=len(vocabulary),
            output_dim=embedding_dims,
            name=f"{feature_name}_embedding",
        )
        # Convert the index values to embedding representations
        encoded_other_features.append(embedding_encoder(idx))

    ## Create a single embedding vector for the user features
    if len(encoded_other_features) > 1:
        encoded_other_features = layers.concatenate(encoded_other_features)
    elif len(encoded_other_features) == 1:
        encoded_other_features = encoded_other_features[0]
    else:
        encoded_other_features = None

    ## Create a movie embedding encoder
    movie_vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY["movie_id"]
    movie_embedding_dims = int(math.sqrt(len(movie_vocabulary)))
    # Create a lookup to convert string values to integer indices
    movie_index_lookup = StringLookup(
        vocabulary=movie_vocabulary,
        mask_token=None,
        num_oov_indices=0,
        name="movie_index_lookup",
    )
    # Create an embedding layer with the specified dimensions
    movie_embedding_encoder = layers.Embedding(
        input_dim=len(movie_vocabulary),
        output_dim=movie_embedding_dims,
        name=f"movie_embedding",
    )
    # Create a vector lookup for movie genres
    genre_vectors = movies[genres].to_numpy()
    movie_genres_lookup = layers.Embedding(
        input_dim=genre_vectors.shape[0],
        output_dim=genre_vectors.shape[1],
        embeddings_initializer=tf.keras.initializers.Constant(genre_vectors),
        trainable=False,
        name="genres_vector",
    )
    # Create a processing layer for genres
    movie_embedding_processor = layers.Dense(
        units=movie_embedding_dims,
        activation="relu",
        name="process_movie_embedding_with_genres",
    )

    ## Define a function to encode a given movie id.
    def encode_movie(movie_id):
        # Convert the string input values into integer indices.
        movie_idx = movie_index_lookup(movie_id)
        movie_embedding = movie_embedding_encoder(movie_idx)
        encoded_movie = movie_embedding
        if include_movie_features:
            movie_genres_vector = movie_genres_lookup(movie_idx)
            encoded_movie = movie_embedding_processor(
                layers.concatenate([movie_embedding, movie_genres_vector])
            )
        return encoded_movie

    ## Encoding target_movie_id
    target_movie_id = inputs["target_movie_id"]
    encoded_target_movie = encode_movie(target_movie_id)

    ## Encoding sequence movie_ids
    sequence_movies_ids = inputs["sequence_movie_ids"]
    encoded_sequence_movies = encode_movie(sequence_movies_ids)
    # Create positional embedding.
    position_embedding_encoder = layers.Embedding(
        input_dim=sequence_length,
        output_dim=movie_embedding_dims,
        name="position_embedding",
    )
    positions = tf.range(start=0, limit=sequence_length - 1, delta=1)
    encodded_positions = position_embedding_encoder(positions)
    # Retrieve sequence ratings to incorporate them into the encoding of the movie.
    sequence_ratings = tf.expand_dims(inputs["sequence_ratings"], -1)
    # Add the positional encoding to the movie encodings and multiply them by rating.
    encoded_sequence_movies_with_poistion_and_rating = layers.Multiply()(
        [(encoded_sequence_movies + encodded_positions), sequence_ratings]
    )

    # Construct the transformer inputs
    for encoded_movie in tf.unstack(
        encoded_sequence_movies_with_poistion_and_rating, axis=1
    ):
        encoded_transformer_features.append(tf.expand_dims(encoded_movie, 1))
    
    encoded_transformer_features.append(encoded_target_movie)
    encoded_transformer_features = layers.concatenate(encoded_transformer_features, axis=1)

    return encoded_transformer_features, encoded_other_features

# BST: Model Specification

In [9]:
include_user_id = False
include_user_features = False
include_movie_features = False

hidden_units = [256, 128]
dropout_rate = 0.1
num_heads = 3


def create_model():
    inputs = create_model_inputs()
    transformer_features, other_features = encode_input_features(
        inputs, include_user_id, include_user_features, include_movie_features
    )

    # Create a multi-headed attention layer
    attention_output = layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=transformer_features.shape[2], dropout=dropout_rate
    )(transformer_features, transformer_features)

    # Transformer block
    attention_output = layers.Dropout(dropout_rate)(attention_output)
    x1 = layers.Add()([transformer_features, attention_output])
    x1 = layers.LayerNormalization()(x1)
    x2 = layers.LeakyReLU()(x1)
    x2 = layers.Dense(units=x2.shape[-1])(x2)
    x2 = layers.Dropout(dropout_rate)(x2)
    transformer_features = layers.Add()([x1, x2])
    transformer_features = layers.LayerNormalization()(transformer_features)
    features = layers.Flatten()(transformer_features)

    # Included the other features
    if other_features is not None:
        features = layers.concatenate(
            [features, layers.Reshape([other_features.shape[-1]])(other_features)]
        )

    # Fully-connected layers
    for num_units in hidden_units:
        features = layers.Dense(num_units)(features)
        features = layers.BatchNormalization()(features)
        features = layers.LeakyReLU()(features)
        features = layers.Dropout(dropout_rate)(features)

    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


model = create_model()

# BST: Training and Evaluation

In [10]:
# Compile the model.
model.compile(
    optimizer=keras.optimizers.Adagrad(learning_rate=0.01),
    loss=keras.losses.MeanSquaredError(),
    metrics=[keras.metrics.MeanAbsoluteError()],
)

# Read the training data.
train_dataset = get_dataset_from_csv("train_data.csv", shuffle=True, batch_size=265)

# Fit the model with the training data.
model.fit(train_dataset, epochs=16)

# Read the test data.
test_dataset = get_dataset_from_csv("test_data.csv", batch_size=265)

# Evaluate the model on the test data.
_, mae = model.evaluate(test_dataset, verbose=0)
print(f"Test MAE: {round(mae, 3)}")

Epoch 1/16
2911/2911 [==============================] - 82s 28ms/step - loss: 1.1600 - mean_absolute_error: 0.8543
Epoch 2/16
2911/2911 [==============================] - 88s 30ms/step - loss: 0.9421 - mean_absolute_error: 0.7715
Epoch 3/16
2911/2911 [==============================] - 88s 30ms/step - loss: 0.8972 - mean_absolute_error: 0.7524
Epoch 4/16
2911/2911 [==============================] - 96s 33ms/step - loss: 0.8742 - mean_absolute_error: 0.7423
Epoch 5/16
2911/2911 [==============================] - 119s 41ms/step - loss: 0.8610 - mean_absolute_error: 0.7362
Epoch 6/16
2911/2911 [==============================] - 150s 52ms/step - loss: 0.8497 - mean_absolute_error: 0.7310
Epoch 7/16
2911/2911 [==============================] - 239s 82ms/step - loss: 0.8405 - mean_absolute_error: 0.7269
Epoch 8/16
2911/2911 [==============================] - 218s 75ms/step - loss: 0.8344 - mean_absolute_error: 0.7247
Epoch 9/16
2911/2911 [==============================] - 159s 55ms/step - los